# Identify and solve topological errors in a polygon layer

## This is a suite of four scripts that find and correct topological errors (gaps and overlaps) in the input polygon layer. 

### Step 1. Find Gaps in the input boundary layer

In [1]:
#by e.kelly/CIESIN - August,2021
# -*- coding: utf-8 -*-
"""
Generated by ArcGIS ModelBuilder on : 2021-08-18 11:53:35. modified by e.kelly
Python 3.7
"""

import arcpy
import numpy
import datetime

In [2]:
#FIND GAPS IN THE INPUT LAYER

# -*- coding: utf-8 -*-
"""
Generated by ArcGIS ModelBuilder on : 2021-08-18 11:53:35. modified by e.kelly

"""

# Model Environment settings
#define your workspace
my_scratch_workspace = r"D:\kellye\DRC_MAN_pnlp\MAN_work.gdb"
my_workspace = r"D:\kellye\DRC_MAN_pnlp\MAN_work.gdb"
OutputLocation = my_workspace

#this is the input poygon layer you want to clean of topological errors
INPUT_BOUNDARY_LAYER = fr"{OutputLocation}\man_ha_joinedAttr_20221123"
#output directory where the gaps layer will be saved
OUTPUT_GAPS = fr"{OutputLocation}\gaps"



In [3]:

def FindAllGaps():  # Find All Gaps in Boundary Layer
   
    # To allow overwriting outputs change overwriteOutput option to True.
    arcpy.env.overwriteOutput = True

    # Model Environment settings
    with arcpy.EnvManager(scratchWorkspace=my_scratch_workspace, workspace=my_workspace):

        # Process: Feature To Polygon (Feature To Polygon) (management)
        FeatureToPolygon = fr"{my_scratch_workspace}\FeatToPoly"
        with arcpy.EnvManager(scratchWorkspace=my_scratch_workspace, workspace=my_workspace):
            arcpy.management.FeatureToPolygon(in_features=INPUT_BOUNDARY_LAYER, out_feature_class=FeatureToPolygon, cluster_tolerance="", attributes="ATTRIBUTES", label_features="")

        # Process: Erase (Erase) (analysis)
        multipart_gaps = fr"{my_scratch_workspace}\multipart_gaps"
        with arcpy.EnvManager(scratchWorkspace=my_scratch_workspace, workspace=my_workspace):
            arcpy.analysis.Erase(in_features=FeatureToPolygon, erase_features=INPUT_BOUNDARY_LAYER, out_feature_class=multipart_gaps, cluster_tolerance="")

        # Process: Multipart To Singlepart (Multipart To Singlepart) (management)
     
        arcpy.management.MultipartToSinglepart(in_features=multipart_gaps, out_feature_class=OUTPUT_GAPS)

if __name__ == '__main__':
    FindAllGaps()


## Step 2. Clean selected gaps. 

### In ArcMap, open the gaps layer generated in Step 1. Select those gaps you want to eliminate and save those as a separate layer. The directory of this layer fill be entered next to "GAPS_TO_BE_FILLED"

### The selected gaps usually have a small area of 500 sqm or less. These gaps do not need any further exploration or confirmation from the country

In [4]:
#verify if the script can be run on the input layer
#error messgage will be printed only if the input layer has the wrong fields

myField = "RefField"

fieldList = arcpy.ListFields(INPUT_BOUNDARY_LAYER)
for field in fieldList:
    if field.name == myField:
        print('RefField already exist; Delete the Field "RefField" in the input layer before running the next script')
        arcpy.management.DeleteField(in_table=INPUT_BOUNDARY_LAYER, drop_field=["RefField"])
        print('Deleted the Field "RefField" in the input layer')
    elif field.name != 'RefField':
        print('There is no RefField in the attribute table')
        break
 
 




There is no RefField in the attribute table


In [5]:
#CLEAN SELECTED GAPS


# -*- coding: utf-8 -*-
"""
Generated by ArcGIS ModelBuilder on : 2021-08-18 11:52:36. modified by e.kelly

"""

import arcpy


# Model Environment settings
my_scratch_workspace = r"D:\kellye\DRC_MAN_pnlp\MAN_work.gdb"
my_workspace = r"D:\kellye\DRC_MAN_pnlp\MAN_work.gdb"
OutputLocation = my_workspace
INPUT_BOUNDARY_LAYER = fr"{my_workspace}\man_ha_joinedAttr_20221123"  #input layer to be cleaned of gaps
GAPS_TO_BE_FILLED = fr"{my_workspace}\gaps"   #these are the gaps we selected and we want to clean
Layer_no_Gaps = fr"{my_workspace}\Layer_no_Gaps"      #output; cleaned layer that has no gaps


def Fill_the_Gaps():  # CleanSelectedGaps

    # To allow overwriting outputs change overwriteOutput option to True.
    arcpy.env.overwriteOutput = True

    # Model Environment settings
    with arcpy.EnvManager(scratchWorkspace=my_scratch_workspace, workspace=my_workspace):
    # Process: (Add Field) (management)

        try:
            FieldAdded = arcpy.management.AddField(in_table=INPUT_BOUNDARY_LAYER, field_name="RefField", field_type="TEXT", field_precision=None, field_scale=None, field_length=20, field_alias="", field_is_nullable="NULLABLE", field_is_required="NON_REQUIRED", field_domain="")
        except arcpy.ExecuteError: 
            # Get the tool error messages 
            msgs = arcpy.GetMessages(2) 

            # Return tool error messages for use with a script tool 
            arcpy.AddError(msgs) 

        # Print tool error messages for use in Python
            print(msgs)

        # Process: (Add Field) (management)
        #with arcpy.EnvManager(scratchWorkspace=my_scratch_workspace, workspace=my_workspace):
        FieldAdded = arcpy.management.AddField(in_table=INPUT_BOUNDARY_LAYER, field_name="RefField", field_type="TEXT", field_precision=None, field_scale=None, field_length=20, field_alias="", field_is_nullable="NULLABLE", field_is_required="NON_REQUIRED", field_domain="")

        # Process: (Calculate Field) (management)
        #with arcpy.EnvManager(scratchWorkspace=my_scratch_workspace, workspace=my_workspace)
        
        Input_Table_with_RefField = arcpy.management.CalculateField(in_table=FieldAdded, field="RefField", expression="'ha'", expression_type="PYTHON3", code_block="", field_type="TEXT", enforce_domains="NO_ENFORCE_DOMAINS")
            
        # Process: Update (Update) (analysis)
        UpdatedWithGaps = fr"{my_scratch_workspace}\healtharea_Update_with_Gaps"
        #with arcpy.EnvManager(scratchWorkspace=my_scratch_workspace, workspace=my_workspace):
        arcpy.analysis.Update(in_features=Input_Table_with_RefField, update_features=GAPS_TO_BE_FILLED, out_feature_class=UpdatedWithGaps, keep_borders="BORDERS", cluster_tolerance="")

        # Process: Select Layer By Attribute (Select Layer By Attribute) (management)
        #with arcpy.EnvManager(scratchWorkspace=my_scratch_workspace, workspace=my_workspace):
        SelectedGaps = arcpy.management.SelectLayerByAttribute(in_layer_or_view=UpdatedWithGaps, selection_type="NEW_SELECTION", where_clause="RefField <> 'ha'", invert_where_clause="")

        # Process: Eliminate (Eliminate) (management)
       
        #with arcpy.EnvManager(scratchWorkspace=my_scratch_workspace, workspace=my_workspace):
        arcpy.management.Eliminate(in_features=SelectedGaps, out_feature_class=Layer_no_Gaps, selection="LENGTH", ex_where_clause="", ex_features="")
        arcpy.management.DeleteField(in_table=Layer_no_Gaps, drop_field=["RefField"])


if __name__ == '__main__':
    Fill_the_Gaps()


## Step 3. Find the overlaps in the layer that has been cleaned of Gaps in the previous step

### The output layer from the previous step (layer cleaned of gaps) will be the input layer here.

In [2]:

# -*- coding: utf-8 -*-
"""
Generated by ArcGIS ModelBuilder on : 2021-08-18 15:32:31. modified by e.kelly
"""
import arcpy

# Model Environment settings
my_scratch_workspace = r"D:\kellye\DRC_MAN_pnlp\MAN_work.gdb"
my_workspace = r"D:\kellye\DRC_MAN_pnlp\MAN_work.gdb"
OutputLocation = r"D:\kellye\DRC_MAN_pnlp\MAN_work.gdb"

#This input layer will be the output from the previous step: the boundary layer cleaned of gaps
INPUT_BOUNDARY_LAYER = fr"{my_workspace}\Layer_no_Gaps"
#Output directory where the overlaps will be saved
Overlaps_Singlepart = fr"{my_workspace}\Overlaps"


In [3]:
#verify if the input layer has a "RefField" column. If yes, this column will be deleted and then added again

myField = "RefField"

fieldList = arcpy.ListFields(INPUT_BOUNDARY_LAYER)
for field in fieldList:   
    if field.name == myField:
        arcpy.management.DeleteField(INPUT_BOUNDARY_LAYER, myField)
    else:
        print('There is no RefField in the attribute table')
        break
        




There is no RefField in the attribute table


In [4]:
#FIND OVERLAPS IN THE INPUT LAYER

def FindOverlaps():  # FindOverlaps

    # To allow overwriting outputs change overwriteOutput option to True.
    arcpy.env.overwriteOutput = True

    # Model Environment settings
    with arcpy.EnvManager(scratchWorkspace=my_scratch_workspace, workspace=my_workspace):
        
        # Process: Intersect (Intersect) (analysis)
        Multipart_Overlaps = fr"{my_scratch_workspace}\Multipart_Overlaps"
        arcpy.analysis.Intersect(in_features=INPUT_BOUNDARY_LAYER, out_feature_class=Multipart_Overlaps, join_attributes="ALL", cluster_tolerance="", output_type="INPUT")

        # Process: Multipart To Singlepart (Multipart To Singlepart) (management)
        arcpy.management.MultipartToSinglepart(in_features=Multipart_Overlaps, out_feature_class=Overlaps_Singlepart)

       
if __name__ == '__main__':
    FindOverlaps()


## Step 4. Clean selected overlaps

### In ArcMap, open the overlaps layer generated in Step 3. Select those overlaps you want to eliminate and save those as a separate layer. The directory of this layer fill be entered next to "Overlaps_Singlepart"

In [5]:
# -*- coding: utf-8 -*-
"""
Generated by ArcGIS ModelBuilder on : 2021-08-18 15:32:31. modified by e.kelly
"""

# Model Environment settings
my_scratch_workspace = r"D:\kellye\DRC_MAN_pnlp\MAN_work.gdb"
my_workspace = r"D:\kellye\DRC_MAN_pnlp\MAN_work.gdb"
OutputLocation = my_workspace

#This input layer will be the output from Step 2: the boundary layer cleaned of gaps
Input_Layer = fr"{my_workspace}\Layer_no_Gaps"
# second input layer: those small-area overlaps, which we want to eliminate
Overlaps_Singlepart =fr"{my_workspace}\Overlaps"
#output directory where the final layer, which has been cleaned of gaps and overlaps will be saved
Layer_Without_Overlaps = fr"{my_workspace}\Layer_no_GapsOverlaps"

In [6]:
#CLEAN THE INPUT LAYER OF OVERLAPS

def CleanOverlaps():  # Clean Overlaps in Boundary Layer

    # To allow overwriting outputs change overwriteOutput option to True.
    arcpy.env.overwriteOutput = True

    # Model Environment settings
    with arcpy.EnvManager(scratchWorkspace=my_scratch_workspace, workspace=my_workspace):

        # Process: Erase (Erase) (analysis)
        OverlapErased = fr"{my_scratch_workspace}\OverlapErased"
        with arcpy.EnvManager(scratchWorkspace=my_scratch_workspace, workspace=my_workspace):
            arcpy.analysis.Erase(in_features=Input_Layer, erase_features=Overlaps_Singlepart, out_feature_class=OverlapErased, cluster_tolerance="")
            arcpy.management.AddField(in_table=OverlapErased, field_name="RefField", field_type="TEXT", field_precision=None, field_scale=None, field_length=None, field_alias="", field_is_nullable="NULLABLE", field_is_required="NON_REQUIRED", field_domain="")
        
        # Process: Calculate Field (2) (Calculate Field) (management)
        with arcpy.EnvManager(scratchWorkspace=my_scratch_workspace, workspace=my_workspace):
            FieldAdded = arcpy.management.AddField(in_table=Overlaps_Singlepart, field_name="RefField", field_type="TEXT", field_precision=None, field_scale=None, field_length=None, field_alias="", field_is_nullable="NULLABLE", field_is_required="NON_REQUIRED", field_domain="")
            OVERLAPS = arcpy.management.CalculateField(in_table=FieldAdded, field="RefField", expression="'overlap'", expression_type="PYTHON3", code_block="", field_type="TEXT", enforce_domains="NO_ENFORCE_DOMAINS")

        
        # Process: Update (Update) (analysis)
        UpdatedWithOverlaps = fr"{my_scratch_workspace}\UpdatedWithOverlaps"
        with arcpy.EnvManager(scratchWorkspace=my_scratch_workspace, workspace=my_workspace):
            arcpy.analysis.Update(in_features=OverlapErased, update_features=OVERLAPS, out_feature_class=UpdatedWithOverlaps, keep_borders="BORDERS", cluster_tolerance="")

        # Process: Select Layer By Attribute (Select Layer By Attribute) (management)
        with arcpy.EnvManager(scratchWorkspace=my_scratch_workspace, workspace=my_workspace):
            Selected_Overlaps = arcpy.management.SelectLayerByAttribute(in_layer_or_view=UpdatedWithOverlaps, selection_type="NEW_SELECTION", where_clause="RefField ='overlap'", invert_where_clause="")

        # Process: Eliminate (Eliminate) (management)
        
        with arcpy.EnvManager(scratchWorkspace=my_scratch_workspace, workspace=my_workspace):
            arcpy.management.Eliminate(in_features=Selected_Overlaps, out_feature_class=Layer_Without_Overlaps, selection="LENGTH", ex_where_clause="", ex_features="")

        # Process: Delete Field (Delete Field) (management)
        Layer_no_Overlaps = arcpy.management.DeleteField(in_table=Layer_Without_Overlaps, drop_field=["RefField"])

if __name__ == '__main__':
    CleanOverlaps()
